## Elliptic Curves

In [12]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.asymmetric.utils import decode_dss_signature
from cryptography.hazmat.primitives.serialization import (
    Encoding, ParameterFormat, PublicFormat,
    load_pem_parameters, load_pem_public_key
)

## ECDSA

ECDSA is a Digital Signature Algorithm. 

We use NIST curve SECP384r1. The key size is 384 bits.
There is no need to generate parameters, because the standard already provides the curve equation, a generator and its order.

We proceed with the generation of a private key and of the corresponding public key.

In [3]:
private_key = ec.generate_private_key( ec.SECP384R1() )
print("Private key is: %d" % private_key.private_numbers().private_value)

Private key is: 33635704155717693405353620886293070477745946902130715807204371010321656843010542517516917047316355171352655336034740


In [4]:
# Alternatively we can generate a key from an integer number (eg 1234) by using
# private_key = ec.derive_private_key( 1234, ec.SECP384R1() )

In [5]:
public_key = private_key.public_key()
print("Public key is : (x = %s, y = %s)" % (public_key.public_numbers().x,public_key.public_numbers().y))

Public key is : (x = 21263400085053286231367251025669544792039179980223644345493395480206074336308964713630426595512819598350299752861949, y = 5441330656277392332567791299355424278499640228179736378409126419609263799624126099855761639354640175647738023353457)


Remember that the private key is an integer, while the public key is a point. Now we can sign a message.
The signature will be encoded using the DER rules, so it will be a string of bits.

In [6]:
data = b"this is some data I'd like to sign"
signature = private_key.sign(
    data,
    ec.ECDSA(hashes.SHA256())
    )
print(signature.hex())

3065023100891e87c0d49e94d640d4425c6fcd8a8f38737589ca448b2ca0830002ed2bab341f427a7d96d752c62232c70783398f0a0230582f0ce0f0d05610bdeef2d8fdc10c8d523aa88cca3037f17a27fd524e03eda028a2ff9c6f28c40cd9570eceb6435e7c


The DSA signature is randomized, so if we run the algorithm again we get a different signature. The interface does not ask for a nonce, so we cannot make the mistake of providing repeated nonces.

In [7]:
signature = private_key.sign(
    data,
    ec.ECDSA(hashes.SHA256())
    )
print(signature.hex())

30640230151ba8a8dd4ade8486df61b591e823f8420fa24a7d677c164ab41a05abdf68e35b15815448db4b33148976c40d5f3bc902307bbe53fbd31e258a098790ceb23d0bc92fe71b429bfdb6f1608bee652c7429a400fdc6e14cde80829747be7d7ad979b0


We can decode the signature and retrieve the parameters $r$ and $s$

In [8]:
(r,s)=decode_dss_signature(signature)
print("r = %d\ns = %d" % (r,s) )

r = 3248825051445311210727916863346667328660788145739950405075486158747012988304000235783552321632121300395825704156105
s = 19045863015172608162317729194239143865899170147418674052844195550884758407551556669873259350475778252494769103665584


We can verify the signature using the public key. If the signature is wrong, we get an exception

In [9]:
try:
    public_key.verify(signature,data,ec.ECDSA(hashes.SHA256()))
    print("OK")
except:
    print("KO")

OK


## Lab

Find a classmate. One plays as Alice, the other plays as Bob.
Alice and Bob generate their keys from secret numbers and have published the corresponding public key.

Alice must prove to Bob that she is Alice.
She derives her private key from her secret number and signs a message.
Then she sends to Bob the message and the signature.
Bob verifies the signature using Alice's public key.

Bob proves his identity to Alice in the same way.

In [25]:
# Alice
Alice_secret_number = 1234
Alice_serialized_public_key = b'-----BEGIN PUBLIC KEY-----\nMHYwEAYHKoZIzj0CAQYFK4EEACIDYgAEe2FShOUqdr8UOfiRMoGoEdS7PY4quCzZ\nprCsAxK1ft1/zoRprQ4Jsxul09T4aB2hG32SVNtusdPAn8ziDFnNH4rCtVJedReL\nURy3R3RY1EqiZ/y6ZLLo7NWdnbXWvQ7C\n-----END PUBLIC KEY-----\n'
Alice_message = b"I'm Alice"

# You can load the other party's public key with
# remote_public_key = load_pem_public_key( serialized_public_key )

In [26]:
# Bob
Bob_secret_number = 4321
Bob_serialized_public_key = b'-----BEGIN PUBLIC KEY-----\nMHYwEAYHKoZIzj0CAQYFK4EEACIDYgAENyisOoDb9lce4qW9DP+LueJUDXeCm89Y\n5ZYW0sx50UdwMwgWWk+HMNfct+tBW9AY2jqMRCPJgyIu9BucU38JuwchT6gsZrQi\nRJTXpndZMbKHD6j31yyaSZ2j9nGxNXkO\n-----END PUBLIC KEY-----\n'
Bob_message = b"I'm Bob"

# You can load the other party's public key with
# remote_public_key = load_pem_public_key( serialized_public_key )